In [1]:
import importlib

if importlib.util.find_spec('transformers') is None:
    print('Installing transformers...')
    ! pip install transformers
    ! pip install pyctcdecode
    ! pip install https://github.com/kpu/kenlm/archive/master.zip
else:
    print('libraries are already installed')

libraries are already installed


In [2]:
from pathlib import Path
# from scipy.io import wavfile
# import scipy.signal
import pandas as pd
# from tqdm.auto import tqdm
# import seaborn as sns
# import matplotlib.pyplot as plt
# from collections import Counter
import numpy as np
# import os
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Reshape
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.regularizers import l1, l2
from transformers import AutoProcessor, TFWav2Vec2Model
# from tensorflow.keras import mixed_precision
from sklearn.metrics import classification_report
import json

## Mount Drive

In [3]:
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ON_COLAB = is_running_on_colab()
ON_COLAB

True

In [4]:
if ON_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  intermediate_folder = Path('/content/gdrive/MyDrive/Colab Notebooks/Speech recognition')
  # intermediate_folder = Path('/content/gdrive/MyDrive/Temp/Speech recognition project')
else:
  intermediate_folder = Path('..') / 'data' / 'intermediate'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Read and define model

In [5]:
SAMPLE_RATE = 16000

In [6]:
# model_name = 'jonatasgrosman/wav2vec2-large-xlsr-53-english'
model_name = 'facebook/wav2vec2-base-960h'
processor = AutoProcessor.from_pretrained(model_name)
model_hf = TFWav2Vec2Model.from_pretrained(model_name)


TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2Model were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

In [7]:
# with open('model_config_after_wav.json') as json_file:
#     json_config = json_file.read()
# model_finetune = tf.keras.models.model_from_json(json_config)
# del json_config

model_finetune = Sequential([
    Conv2D(4, (49, 1), activation='relu', padding='same', input_shape=(49, 768, 1)),
    MaxPooling2D((49, 1)),
    # Conv2D(2, (3, 1), activation='relu', padding='same'),
    # MaxPooling2D((3, 1)),
    # Conv2D(2, (3, 1), activation='relu', padding='same'),
    # MaxPooling2D((3, 1)),
    Flatten(),
    Dropout(.5),
    # Dense(32, activation='relu'),
    # Dropout(.5),
    # Dense(128, activation='relu'),
    # Dropout(.2),
    Dense(20, activation='softmax')])

In [8]:
model_finetune.load_weights((intermediate_folder / 'best_model_after_wav.ckp'))

In [9]:
input_ids = tf.keras.Input(shape=(SAMPLE_RATE,), dtype='float64', name='input')
attention_masks = tf.keras.Input(shape=(SAMPLE_RATE,), dtype='int32', name='mask')
output = model_hf([input_ids, attention_masks])
output = output[0]
output = model_finetune(output)
model_full = tf.keras.models.Model(inputs={"input": input_ids, "mask": attention_masks}, outputs=output)
model_full.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input[0][0]',                  
 Model)                         Output(last_hidden_               'mask[0][0]']                   
                                state=(None, 49, 76                                               
                                8),                                                           

## Read data

In [10]:
BATCH_SIZE = 256

In [11]:
# X_train = np.load(intermediate_folder / 'train_main_1_sec_audio.npy')
# X_train.shape

In [12]:
# X_train_input = processor(X_train, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
# del X_train
# X_train_input.shape

In [13]:
# X_val = np.load(intermediate_folder / 'val_main_1_sec_audio.npy')
# X_val.shape

In [14]:
# X_val_input = processor(X_val, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
# del X_val
# X_val_input.shape

In [15]:
X_test = np.load(intermediate_folder / 'test_main_1_sec_audio.npy')
X_test.shape

(4689, 16000)

In [16]:
X_test_input = processor(X_test, return_tensors="tf", sampling_rate=SAMPLE_RATE).input_values
del X_test
X_test_input.shape

TensorShape([4689, 16000])

In [17]:
# y_train_labels = pd.read_csv(intermediate_folder / 'train_main_1_sec_labels.csv', header=None, index_col=False)[0]
# y_train_labels.shape

In [18]:
# y_val_labels = pd.read_csv(intermediate_folder / 'val_main_1_sec_labels.csv', header=None, index_col=False)[0]
# y_val_labels.shape

In [19]:
y_test_labels = pd.read_csv(intermediate_folder / 'test_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_test_labels.shape

(4689,)

In [20]:
with open(intermediate_folder / 'labels.json', 'r') as file:
    labels_dict = json.load(file)
labels_dict

{'down': 0,
 'eight': 1,
 'five': 2,
 'four': 3,
 'go': 4,
 'left': 5,
 'nine': 6,
 'no': 7,
 'off': 8,
 'on': 9,
 'one': 10,
 'right': 11,
 'seven': 12,
 'six': 13,
 'stop': 14,
 'three': 15,
 'two': 16,
 'up': 17,
 'yes': 18,
 'zero': 19}

In [21]:
# le = LabelEncoder()
# le.fit(y_train_labels)

# y_train = le.transform(y_train_labels)
# y_val = le.transform(y_val_labels)
# y_test = le.transform(y_test_labels)

# del y_test_labels
# del y_train_labels
# del y_val_labels
# y_test.shape

In [22]:
# y_train = [labels_dict(lbl) for lbl in y_train_labels]
# y_val = [labels_dict(lbl) for lbl in y_val_labels]
y_test = [labels_dict[lbl] for lbl in y_test_labels]

# del y_train_labels
# del y_val_labels
del y_test_labels

In [23]:
# import json
# classes = {cl: i for i, cl in enumerate(le.classes_)}
# print(classes)
# with open(intermediate_folder / 'labels.json', 'w') as fp:
#     json.dump(classes, fp)

## Predict and metrics

In [24]:
# y_pred = model_full.predict([X_train_input, tf.ones([len(X_train_input), SAMPLE_RATE], dtype=tf.int32)], batch_size=BATCH_SIZE)
# print((y_pred.argmax(axis=1) == y_train).mean())
# print(classification_report(y_train, y_pred.argmax(axis=1)))

In [25]:
# y_pred = model_full.predict([X_val_input, tf.ones([len(X_val_input), SAMPLE_RATE], dtype=tf.int32)], batch_size=BATCH_SIZE)
# print((y_pred.argmax(axis=1) == y_val).mean())
# print(classification_report(y_val, y_pred.argmax(axis=1)))

In [26]:
y_pred = model_full.predict([X_test_input, tf.ones([len(X_test_input), SAMPLE_RATE], dtype=tf.int32)], batch_size=BATCH_SIZE)
print((y_pred.argmax(axis=1) == y_test).mean())
print(classification_report(y_test, y_pred.argmax(axis=1)))

19/19 [==============================] - 47s 2s/step
0.9505224994668373
              precision    recall  f1-score   support

           0       0.95      0.94      0.94       238
           1       0.96      0.92      0.94       230
           2       0.97      0.95      0.96       241
           3       0.93      0.94      0.93       235
           4       0.94      0.92      0.93       223
           5       0.94      0.97      0.95       249
           6       0.95      0.97      0.96       236
           7       0.97      0.95      0.96       233
           8       0.95      0.92      0.93       243
           9       0.91      0.94      0.92       221
          10       0.95      0.94      0.94       222
          11       0.96      0.94      0.95       233
          12       0.97      0.97      0.97       222
          13       0.99      0.98      0.98       231
          14       0.94      0.98      0.96       234
          15       0.94      0.95      0.95       246
         

## Save model

In [27]:
# Save the architecture
with open(intermediate_folder / 'full_model_config.json', 'w') as json_file:
    json_file.write(model_full.to_json())

# Save weights
model_full.save_weights(intermediate_folder / 'full_model_weights.h5')